In [1]:
import tensorflow as tf
from keras.layers import Layer
import numpy as np
import cv2
import warnings
from datetime import datetime
import tkinter as tk
from tkinter import filedialog
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz"
modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz"
# modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz"

In [3]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#     # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#         print(e)

In [4]:
h = 105
w = 105
def preprocess_image(filename):
    img_st = tf.io.read_file(filename)
    img1 = tf.image.decode_jpeg(img_st, channels=3)
    img1 = tf.image.convert_image_dtype(img1, tf.float32)
    img1 = tf.image.resize(img1, (h, w))
    return img1

def preprocess_twin(inp_img, val_img):
    return (preprocess_image(inp_img), preprocess_image(val_img))

class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    def call(self, inp_em, val_em):
        return tf.math.abs(inp_em-val_em)

binary_cross_loss = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4)

In [5]:
# def ForMatch():
#     a_img = ["report\\target\\testv1_frame_69.jpg"]*8
#     t_img = ["report\\group\\new_lg\\testv1_frame_5.jpg",
#             "report\\group\\new_lg\\testv1_frame_64.jpg",
#             "report\\group\\new_lg\\testv1_frame_70.jpg",
#             "report\\group\\new_lg\\testv2_frame_38.jpg"]

#     anchor_img = tf.data.Dataset.from_tensor_slices(a_img)
#     test_img = tf.data.Dataset.from_tensor_slices(t_img)

#     dataset = tf.data.Dataset.zip((anchor_img, test_img))
#     test_dataset = dataset.map(preprocess_twin)
#     test_dataset = test_dataset.batch(8, drop_remainder=False)

#     siamese = tf.keras.models.load_model("weight\\FinalMatchTrainFET-1024-4L-fin.h5", 
#                                         custom_objects = {"L1Dist":L1Dist, 
#                                                         "BinaryCrossentropy":tf.losses.BinaryCrossentropy})

#     data = test_dataset.as_numpy_iterator()
#     test_inp, test_val = data.next()
#     yhat = siamese.predict([test_inp, test_val])
#     print(yhat)
#     # cv2.imwrite(f"./report/result/{imgName[:len(imgName)-4]}__{yhat}.jpg", cv2.imread(f"./detected/{imgName}"))
# ForMatch()

In [6]:
def checkForMatch(imgName):
    # a_img = ["./testQ/t.jpg"]
    a_img = ["./testQ/t.jpg"]
    t_img = [f"./detected/{imgName}"]

    anchor_img = tf.data.Dataset.from_tensor_slices(a_img)
    test_img = tf.data.Dataset.from_tensor_slices(t_img)

    dataset = tf.data.Dataset.zip((anchor_img, test_img))
    test_dataset = dataset.map(preprocess_twin)
    test_dataset = test_dataset.batch(1, drop_remainder=False)

    siamese = tf.keras.models.load_model("weight/NewArch-lr-0.0001-one-bn-1024.h5", 
                                        custom_objects = {"L1Dist":L1Dist, 
                                                        "BinaryCrossentropy":tf.losses.BinaryCrossentropy})

    data = test_dataset.as_numpy_iterator()
    test_inp, test_val = data.next()
    yhat = siamese.predict([test_inp, test_val])

    if yhat > 0.8:
        cv2.imwrite(f"./result/{imgName[:len(imgName)-4]}__{yhat}.jpg", cv2.imread(f"./detected/{imgName}"))
        print(yhat)
        return True
    else:
        return False

In [7]:
from fileinput import filename
import os
from tensorflow.python.keras.utils.data_utils import get_file
np.random.seed(123)

# Create a detector class
class Detector:
    # init method
    def __init__(self):
        pass

    # Read classes from file
    def read_classes(self, path):
        with open(path, 'r') as f:
            self.classList = f.read().splitlines()
        self.colorList = np.random.uniform(
            0, 255, size=(len(self.classList), 3))

    # Download Model
    def download_model(self, url):
        self.fileName = os.path.basename(url)
        self.modelName = self.fileName[:self.fileName.index('.')]

        self.model_dir = './models'
        os.makedirs(self.model_dir, exist_ok=True)

        # get the model
        get_file(fname=self.fileName,
                 origin=url,
                 cache_dir=self.model_dir,
                 cache_subdir=self.modelName,
                 extract=True)

    # Load Model
    def load_model(self):
        # print("Loading model... "+self.modelName)
        print("Loading model... ")
        tf.keras.backend.clear_session()
        model_path = os.path.join(
            self.model_dir, self.modelName, self.modelName, "saved_model")
        # print("Model path: "+model_path)
        self.model = tf.saved_model.load(model_path)
        print("Model loaded")

    # Retrieve bounding boxes and class names
    def createBoundingBoxes(self, image, threshold, timer):
        image_tensor = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = tf.convert_to_tensor(image_tensor) #Changed image to image_tensor
        image_tensor = image_tensor[tf.newaxis, ...]

        results = self.model(image_tensor)

        bboxes = results['detection_boxes'][0].numpy()
        classesIndexes = results['detection_classes'][0].numpy().astype(
            np.int32)
        ClassScores = results['detection_scores'][0].numpy()

        imH, imW, imgC = image.shape
        bboxind = tf.image.non_max_suppression(bboxes,
                                               ClassScores,
                                               max_output_size=20,
                                               iou_threshold=threshold,
                                               score_threshold=threshold)
        if len(bboxind) > 0:
            c = 0
            for i in bboxind:
                bbox = tuple(bboxes[i].tolist())
                classConfidence = round(ClassScores[i] * 100, 2)
                classInd = classesIndexes[i]
                classLabelText = self.classList[classInd]
                if classLabelText != "person":
                    continue
                c += 1
                classColor = self.colorList[classInd]
                displayText = "{}: {}%".format(
                    classLabelText, classConfidence)
                ymin, xmin, ymax, xmax = bbox
                ymin, xmin, ymax, xmax = int(
                    ymin * imH), int(xmin * imW), int(ymax * imH), int(xmax * imW)
                crop_img = image[ymin:ymax,xmin:xmax]
                print(crop_img.shape)
                imgName = f"{timer}_{c}.jpg"
                print(imgName)
                cv2.imwrite(f"./detected/{imgName}", crop_img)
                # _ = checkForMatch(imgName)
                if not checkForMatch(imgName):
                    continue
                cv2.rectangle(image, (xmin, ymin),
                              (xmax, ymax), classColor, 3)

                cv2.imshow('Output Result', image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                        break

                # cv2.putText(image,
                #             displayText,
                #             (xmin, ymin-10),
                #             cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                #             classColor, 2)
        return image

    # Predict person on image
    def predict(self, imagePath, threshold):
        width = 600
        height = 550
        dim = (width, height)
        if imagePath.endswith('.mp4'):
            cap = cv2.VideoCapture(imagePath)

            frame_width = int(cap.get(3))
            frame_height = int(cap.get(4))
            frame_size = (frame_width, frame_height)
            
            # scale_percent = sf # percent of original size
            # width = int(cap.get(3) * scale_percent / 100)
            # height = int(cap.get(4) * scale_percent / 100)

            out_vid = cv2.VideoWriter(
                "result"+'.mp4',
                cv2.VideoWriter_fourcc(*'mp4v'), 30,
                frame_size)
            timer = -1
            while(cap.isOpened()):
                ret, frame = cap.read()
                timer += 1
                if ret == True:
                    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
                    frame = self.createBoundingBoxes(frame, threshold, timer)
                    cv2.imwrite("./FullFrame/" + str(timer) + ".jpg", frame)
                    out_vid.write(frame)

                    # cv2.imshow('Output Result', frame)
                    # if cv2.waitKey(1) & 0xFF == ord('q'):
                    #     break

                else:
                    break
            out_vid.release()
            cap.release()
            # cv2.destroyAllWindows()
        else:
            image = cv2.imread(imagePath)
            image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            timer = 0
            bbimage = self.createBoundingBoxes(image, threshold, timer)
            cv2.imwrite("./FullFrame/" + self.modelName + ".jpg", bbimage)
            cv2.imshow("Result", bbimage)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


In [8]:
def getTargetImage(filePath, typeOfTarget):
    width = 600
    height = 550
    dim = (width, height)
    
    if typeOfTarget=="video":
        # Create a VideoCapture object and read from input file
        cap = cv2.VideoCapture(filePath)

        # scale_percent = sf # percent of original size
        # width = int(cap.get(3) * scale_percent / 100)
        # height = int(cap.get(4) * scale_percent / 100)
        # width = 700
        # height = 600
        # dim = (width, height)
        
        # Check if camera opened successfully
        if (cap.isOpened()== False):
            print("Error opening video file")
        
        # Read until video is completed
        while(cap.isOpened()):
        # Capture frame-by-frame
            ret, frame = cap.read()
            if ret == True:
                frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
            # Display the resulting frame
                cv2.imshow('Frame', frame)
                # Press Q on keyboard to exit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
                # Press F on keyboard to select Image region
                if cv2.waitKey(25) & 0xFF == ord('f'):
                    roi = cv2.selectROI(frame)
                    print(roi)

                    x1,y1,x2,y2 = roi
                    if not x1 and not x2 and not y1 and not y2:
                        cv2.destroyAllWindows()
                    else:
                        crop_img = frame[y1:y1+y2,x1:x1+x2]
                        cv2.imwrite("./testQ/t.jpg", crop_img)

                    cv2.waitKey(100)
                    cv2.destroyAllWindows()
        # Break the loop
            else:
                break
        # When everything done, release
        # the video capture object
        cap.release()
        # Closes all the frames
        cv2.destroyAllWindows()
    else:
        frame = cv2.imread(filePath)
        frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
        # scale_percent = sf # percent of original size
        # width = int(frame.shape[0] * scale_percent / 100)
        # height = int(frame.shape[1] * scale_percent / 100)
        # dim = (width, height)
        roi = cv2.selectROI(frame)
        print(roi)

        x1,y1,x2,y2 = roi
        if not x1 and not x2 and not y1 and not y2:
            cv2.destroyAllWindows()
        else:
            crop_img = frame[y1:y1+y2,x1:x1+x2]
            cv2.imwrite("./testQ/t.jpg", crop_img)

        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [9]:
# print("Chose type of Target:")
# print("If Target is an image insert 0")
# print("If Target is a video insert 1")
# typeOfTarget = int(input("Insert type: "))
# scaleFactor = int(input("Scaling Ratio: "))
# if typeOfTarget == 0: typeOfTarget = "Image"
# else: typeOfTarget = "Video"
# print(typeOfTarget)
# print(scaleFactor)
# print("Chose type of Query:")
# print("If Query is an image insert 0")
# print("If Query is a video insert 1")
# typeOfQuery = int(input("Insert type: "))
# if typeOfQuery == 0: typeOfQuery = "Image"
# else: typeOfQuery = "Video"
# print(typeOfQuery)
# getTargetImage(typeOfTarget,scaleFactor)

In [10]:
import tkinter as tk
from tkinter import filedialog

# -----------------------  GUI CODE  -----------------------------
def open_file1():
    file1_path = filedialog.askopenfilename()
    file1_label.config(text=file1_path)


def open_file2():
    file2_path = filedialog.askopenfilename()
    file2_label.config(text=file2_path)


def get_link():
    file1 = file1_label.cget("text")
    file2 = file2_label.cget("text")
    typeOfTarget = t_Type.get().lower()
    typeOfQuery = q_Type.get().lower()
    print("Target Location:", file1)
    print("Target Type:", typeOfTarget)
    print("Query Location:", file2)
    print("Query Type:", typeOfQuery)
    getTargetImage(file1, typeOfTarget)
    namePath = "Required Files\coco.names"
    # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz"
    # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz"
    # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz"
    # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz"
    # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz"
    # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz"
    modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"
    imagePath = file2
    VideoPath = file2
    threshold = 0.6
    detector = Detector()
    detector.read_classes(namePath)
    detector.download_model(modelURL)
    detector.load_model()
    if typeOfQuery == "video":
        detector.predict(VideoPath, threshold)
    else:
        detector.predict(imagePath, threshold)
    print("Completed")

root = tk.Tk()
root.geometry("800x450+300+150")
root.title("File Selector GUI")

################ Target Select #################

file1_button = tk.Button(root, text="Select Target File to Crop", command=open_file1)
file1_button.pack()

file1_label = tk.Label(root, text="No file selected")
file1_label.pack()

t_Label = tk.Label(root, text="Enter Target Type:")
t_Label.pack()

t_Type = tk.Entry(root)
t_Type.pack()

############### Query Select ###################

file2_button = tk.Button(root, text="Select Query File to Search in", command=open_file2)
file2_button.pack()

file2_label = tk.Label(root, text="No file selected")
file2_label.pack()

q_Label = tk.Label(root, text="Enter Query Type:")
q_Label.pack()

q_Type = tk.Entry(root)
q_Type.pack()

#################### Submit Button ###################################

submit_button = tk.Button(root, text="Submit", command=get_link)
submit_button.pack()

#####################

root.mainloop()

# ----------------------- END OF GUI CODE --------------------------

Target Location: D:/Academic/4th Year/2k17/Thesis/Pendrive Contents/Program/target.mp4
Target Type: video
Query Location: D:/Academic/4th Year/2k17/Thesis/Pendrive Contents/Program/query.mp4
Query Type: video
(125, 137, 125, 372)
20520960/20518283 [==============================] - 24s 1us/step
Loading model... 
Model loaded
(347, 108, 3)
91_1.jpg
1/1 [==============================] - 2s 2s/step
[[0.8090524]]
(336, 107, 3)
92_1.jpg
1/1 [==============================] - 0s 219ms/step
(339, 106, 3)
93_1.jpg
1/1 [==============================] - 0s 226ms/step
(327, 115, 3)
95_1.jpg
1/1 [==============================] - 0s 229ms/step
[[0.95050764]]
(253, 108, 3)
96_1.jpg
1/1 [==============================] - 0s 214ms/step
(246, 105, 3)
99_1.jpg
1/1 [==============================] - 0s 198ms/step
[[0.8053517]]
(251, 106, 3)
100_1.jpg
1/1 [==============================] - 0s 251ms/step
[[0.89521706]]
(249, 110, 3)
101_1.jpg
1/1 [==============================] - 0s 195ms/step
(249, 10

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Zahim\anaconda3\envs\mllab\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-10-b6eee72e97d4>", line 41, in get_link
    detector.predict(VideoPath, threshold)
  File "<ipython-input-7-596e2dae6911>", line 128, in predict
    frame = self.createBoundingBoxes(frame, threshold, timer)
  File "<ipython-input-7-596e2dae6911>", line 51, in createBoundingBoxes
    results = self.model(image_tensor)
  File "c:\Users\Zahim\anaconda3\envs\mllab\lib\site-packages\tensorflow\python\saved_model\load.py", line 686, in _call_attribute
    return instance.__call__(*args, **kwargs)
  File "c:\Users\Zahim\anaconda3\envs\mllab\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\Zahim\anaconda3\envs\mllab\lib\site-packages\tensorflow\python\eager\execute.py", line 55, in q

KeyboardInterrupt: 

: 

In [ ]:
# import tkinter as tk

# root = tk.Tk()
# root.geometry("800x450+300+150")
# root.mainloop()

In [ ]:
# filePath = "Required Files\coco.names"
# # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz"
# # modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz"
# modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"
# imagePath = file
# VideoPath = "./query.mp4"
# threshold = 0.6
# detector = Detector()
# detector.read_classes(filePath)
# detector.download_model(modelURL)
# detector.load_model()
# if typeOfQuery == "Video":
#     detector.predict(VideoPath, threshold, scaleFactor)
# else:
#     detector.predict(imagePath, threshold, scaleFactor)
# print("Completed")

In [ ]:
# import tkinter as tk
# from tkinter import filedialog

# # -----------------------  GUI CODE  -----------------------------


# def open_file1():
#     file1_path = filedialog.askopenfilename()
#     file1_label.config(text=file1_path)


# def open_file2():
#     file2_path = filedialog.askopenfilename()
#     file2_label.config(text=file2_path)


# def get_link():
#     file1 = file1_label.cget("text")
#     file2 = file2_label.cget("text")
#     target = t_Type.get().lower()
#     query = q_Type.get().lower()
#     print("Target Location:", file1)
#     print("Target Type:", target)
#     print("Query Location:", file2)
#     print("Query Type:", query)


# root = tk.Tk()
# root.title("File Selector GUI")

# ########## Target Select #################

# file1_button = tk.Button(root, text="Select Target File", command=open_file1)
# file1_button.pack()

# file1_label = tk.Label(root, text="No file selected")
# file1_label.pack()

# t_Label = tk.Label(root, text="Enter Target Type:")
# t_Label.pack()

# t_Type = tk.Entry(root)
# t_Type.pack()

# ############### Query Select ###################

# file2_button = tk.Button(root, text="Select Query File", command=open_file2)
# file2_button.pack()

# file2_label = tk.Label(root, text="No file selected")
# file2_label.pack()

# q_Label = tk.Label(root, text="Enter Query Type:")
# q_Label.pack()

# q_Type = tk.Entry(root)
# q_Type.pack()

# #################### Submit Button ###################################

# submit_button = tk.Button(root, text="Submit", command=get_link)
# submit_button.pack()

# #####################

# root.mainloop()

# # ----------------------- END OF GUI CODE --------------------------


In [ ]:
# # getQueryImage(fname)
# print("Chose type of Target:")
# print("If Target is an image insert 0")
# print("If Target is a video insert 1")
# typeOfTarget = int(input("Insert type: "))
# scaleFactor = int(input("Scaling Ratio: "))
# print(typeOfTarget)
# print(scaleFactor)
# print("Chose type of Query:")
# print("If Query is an image insert 0")
# print("If Query is a video insert 1")
# typeOfQuery = int(input("Insert type: "))
# print(typeOfQuery)
# getQueryImage(typeOfTarget)